# Multi-Layer Perceptron Demo

In [2]:
from mlp.activations import Sigmoid
from mlp.losses import MeanSquaredError
from mlp.mlp import MLP, MLPLayersBuilder

import numpy as np

## XOR Problem

In [8]:
X = np.array([[0,0],
              [0,1],
              [1,0],
              [1,1]])

y = np.array([[0, 1, 1, 0]]).T

mlp_layers_builder = MLPLayersBuilder()\
    .add_input(2)\
    .add_dense(5, Sigmoid())\
    .add_dense(1, Sigmoid())
layers = mlp_layers_builder.build()

mlp = MLP(layers, loss_function=MeanSquaredError(), n_epochs=10000, learning_rate=0.9)
mlp.fit(X,y)

print(mlp.forward(X))

Epoch 1/10000 | Loss: [0.50020785]
Epoch 1001/10000 | Loss: [0.49999689]
Epoch 2001/10000 | Loss: [0.49998977]
Epoch 3001/10000 | Loss: [0.49997354]
Epoch 4001/10000 | Loss: [0.4999154]
Epoch 5001/10000 | Loss: [0.49932464]
Epoch 6001/10000 | Loss: [0.43353927]
Epoch 7001/10000 | Loss: [0.15922151]
Epoch 8001/10000 | Loss: [0.02109562]
Epoch 9001/10000 | Loss: [0.00881177]
[[0.04675456]
 [0.95023446]
 [0.950628  ]
 [0.05898119]]
